## Requirements

### Imports

In [1]:
import os
import shutil
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [ ]:
torch.manual_seed(42)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Paths

In [4]:
src_folder = Path.cwd()
root_folder = src_folder.parent
dataset_folder = os.path.join(root_folder, 'dataset')
models_folder = os.path.join(root_folder, 'models')

custom_folder = os.path.join(models_folder, 'custom')
result_folder = os.path.join(custom_folder, 'results')

PREPROCESSED_DIR = os.path.join(dataset_folder, 'preprocessed')
CSV_PATH = os.path.join(dataset_folder, 'csv_mappings', 'train.csv')

## Config

In [5]:
BATCH_SIZE = 32
NUM_CLASSES = 10 
EPOCHS = 20
PATIENCE = 3
LEARNING_RATE = 0.0001

## Data Preperation

### Mushroom Dataset

In [6]:
class MushroomDataset(Dataset):
    def __init__(self, preprocessed_dir, csv_path, transform=None):
        self.preprocessed_dir = preprocessed_dir  
        self.csv_path = csv_path  
        self.transform = transform  
        self.csv_data = pd.read_csv(csv_path)
        
        # Images and Labels
        self.image_ids = self.csv_data['Image'].values  
        self.labels = self.csv_data['Mushroom'].values 
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        label = self.labels[idx]

        image_id_str = str(image_id).zfill(5)  # Pad for filename
        
        # Load .pt files
        image_path = os.path.join(self.preprocessed_dir, f"{image_id_str}.pt")
        image = torch.load(image_path)  
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


### Prepare data

In [7]:
dataset = MushroomDataset(PREPROCESSED_DIR, CSV_PATH)
indices = list(range(len(dataset)))

### Split data

In [8]:
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

In [9]:
train_subset = torch.utils.data.Subset(dataset, train_indices)
val_subset = torch.utils.data.Subset(dataset, val_indices)
test_subset = torch.utils.data.Subset(dataset, test_indices)

### Dataloaders

In [10]:
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False)

## Train & Test functions

In [11]:
def save_model(model, optimizer, epoch, loss, accuracy, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'accuracy': accuracy,
    }

    torch.save(checkpoint, file_path)
    print(f"Model saved to {file_path}")

In [12]:
def train_on_epoch(model, train_loader, criterion, optimizer, device, epoch, writer, scheduler):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for batch_idx, data in enumerate(tqdm(train_loader, desc="[Train]")):
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Log loss and accuracy per batch to TensorBoard
        batch_accuracy = 100.0 * correct / total
        writer.add_scalar('Train/Loss', loss.item(), epoch * len(train_loader) + batch_idx)
        writer.add_scalar('Train/Accuracy', batch_accuracy, epoch * len(train_loader) + batch_idx)

    train_accuracy = 100.0 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    # Log the learning rate
    for param_group in optimizer.param_groups:
        writer.add_scalar('Train/Learning Rate', param_group['lr'], epoch)

    # Step the scheduler
    if scheduler:
        scheduler.step()

    return avg_train_loss, train_accuracy

In [13]:
def validate_on_epoch(model, val_loader, criterion, optimizer, device, epoch, writer, best_val_loss, patience, epochs_no_improve, save_path):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, data in enumerate(tqdm(val_loader, desc="[Val]")):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Log loss and accuracy per batch to TensorBoard
            batch_accuracy = 100.0 * correct / total
            writer.add_scalar('Validation/Loss', loss.item(), epoch * len(val_loader) + batch_idx)
            writer.add_scalar('Validation/Accuracy', batch_accuracy, epoch * len(val_loader) + batch_idx)

    val_accuracy = 100.0 * correct / total
    avg_val_loss = val_loss / len(val_loader)

    # Early stopping 
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        save_model(model, optimizer, epoch, avg_val_loss, val_accuracy, save_path)
        print(f"Model saved to {save_path}")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered at epoch {epoch + 1}")
        return best_val_loss, epochs_no_improve, True, val_accuracy 

    return best_val_loss, epochs_no_improve, False, val_accuracy

In [14]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs, device, writer, scheduler, patience, save_path):
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Train
        train_loss, train_accuracy = train_on_epoch(model, train_loader, criterion, optimizer, device, epoch, writer, scheduler)
        print(f"Train Loss = {train_loss:.4f}, Train Acc = {train_accuracy:.2f}%")

        # Validate
        best_val_loss, epochs_no_improve, early_stop, val_accuracy = validate_on_epoch(
            model, val_loader, criterion, optimizer, device, epoch, writer, best_val_loss, patience, epochs_no_improve, save_path
        )
        print(f"Val Loss = {best_val_loss:.4f}, Val Acc = {val_accuracy:.2f}%")

        if early_stop:
            break  

    return model

In [15]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for data in tqdm(test_loader, desc="[Test]"):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Store preds and labels for plots
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_accuracy = 100.0 * correct / total
    avg_test_loss = test_loss / len(test_loader)

    print(f"Test Loss = {avg_test_loss:.4f}")
    print(f"Test Accuracy = {test_accuracy:.2f}%")
    
    return avg_test_loss, test_accuracy, all_labels, all_predictions


In [16]:
def plot_confusion_matrix(all_labels, all_predictions, num_classes, save_path=None):
    conf_matrix = confusion_matrix(all_labels, all_predictions, labels=np.arange(num_classes))
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
                xticklabels=np.arange(num_classes), yticklabels=np.arange(num_classes))
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
        print(f"Confusion matrix saved to {save_path}")

    plt.show()


In [17]:
def per_class_accuracy(all_labels, all_predictions, num_classes):
    class_accuracies = []
    
    for i in range(num_classes):
        class_indices = [j for j, label in enumerate(all_labels) if label == i]
        class_predictions = [all_predictions[j] for j in class_indices]
        class_labels = [all_labels[j] for j in class_indices]
        
        class_accuracy = accuracy_score(class_labels, class_predictions)
        class_accuracies.append(class_accuracy)
        print(f"Accuracy class {i}: {class_accuracy:.4f}")
    
    return class_accuracies

In [18]:
def display_classification_report_as_dataframe(all_labels, all_predictions):
    report_dict = classification_report(all_labels, all_predictions, target_names=[str(i) for i in range(len(set(all_labels)))], output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    display(report_df)
    return report_df

# Training

#### Custom Model

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class EnhancedResNet(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0.5):
        super(EnhancedResNet, self).__init__()

        self.resnet = models.resnet50(pretrained=True)  
        self.resnet.fc = nn.Identity()
        self.dropout = nn.Dropout(p=dropout_prob)
        
        self.fc1 = nn.Linear(2048, 1024)  
        self.fc2 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.resnet(x)  
        x = self.dropout(x) 
        x = torch.relu(self.fc1(x)) 
        x = self.fc2(x)  
        return x


In [ ]:
# Define Enhanced AlexNet Model
class EnhancedAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(EnhancedAlexNet, self).__init__()
        # Feature extraction layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),  # Layer 1
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),             # Layer 2
            nn.Conv2d(64, 192, kernel_size=5, padding=2),         # Layer 3
            nn.BatchNorm2d(192),                       # BatchNorm added
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),             # Layer 4
            nn.Conv2d(192, 384, kernel_size=3, padding=1),        # Layer 5
            nn.BatchNorm2d(384),                       # BatchNorm added
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),        # Layer 6
            nn.BatchNorm2d(256),                       # BatchNorm added
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),        # Layer 7
            nn.BatchNorm2d(256),                       # BatchNorm added
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),             # Layer 8
        )

        # Pooling layer to ensure consistent output size
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        # Classifier layers with Dropout for regularization
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),                    # Dropout 1
            nn.Linear(256 * 6 * 6, 4096),            # Fully Connected Layer 1
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),                    # Dropout 2
            nn.Linear(4096, 4096),                  # Fully Connected Layer 2
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),               # Fully Connected Layer 3
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)  # Flatten for fully connected layers
        x = self.classifier(x)
        return x

In [20]:
from torch.optim.lr_scheduler import OneCycleLR

def get_optimizer_and_scheduler(model, lr=0.001, weight_decay=1e-5, momentum=0.9):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # scheduler = StepLR(optimizer, step_size=2, gamma=0.5)
    scheduler = OneCycleLR(
        optimizer,
        max_lr=lr,  
        steps_per_epoch=len(train_loader),
        epochs=EPOCHS,
        pct_start=0.3,
        anneal_strategy='cos',
        div_factor=25.0,
        final_div_factor=10000.0
    )
    
    return optimizer, scheduler

#### Setter for model 

In [21]:
def set_model_for_training(model_type):
    base_log_path = os.path.join(custom_folder, model_type, 'log')
    base_result_path = os.path.join(custom_folder, model_type, 'results')

    if os.path.exists(base_log_path):
        shutil.rmtree(base_log_path)
    os.makedirs(base_log_path, exist_ok=True)
    os.makedirs(base_result_path, exist_ok=True)

    if model_type == 'custom':
        model = EnhancedResNet(num_classes=NUM_CLASSES) # EnhancedAlexNet
        save_path = os.path.join(base_result_path, "model_custom.pth")

    else:
        raise ValueError(f"Unsupported model type")
    
    writer = SummaryWriter(log_dir=base_log_path)

    return model, save_path, writer


## Training

In [22]:
model_type = 'custom' 

In [ ]:
model, save_path, writer = set_model_for_training(model_type)
criterion = nn.CrossEntropyLoss()

In [24]:
optimizer, scheduler = get_optimizer_and_scheduler(model, lr=0.001, weight_decay=1e-5, momentum=0.9)

In [ ]:
model = train_and_validate(model, 
                           train_loader, 
                           val_loader, 
                           criterion, 
                           optimizer, 
                           epochs=EPOCHS, 
                           device=device, 
                           writer=writer, 
                           scheduler=scheduler, 
                           patience=PATIENCE, 
                           save_path=save_path)
writer.close()

## Evaluation

In [25]:
evaluation_results_path = os.path.join(os.path.dirname(save_path), "evaluation_results.txt")
confusion_matrix_path = os.path.join(os.path.dirname(save_path), "plot_confusion_matrix.png")

In [ ]:
avg_test_loss, test_accuracy, all_labels, all_predictions = evaluate_model(model, test_loader, criterion, device)

In [ ]:
per_class_acc = per_class_accuracy(all_labels, all_predictions, NUM_CLASSES)

In [ ]:
report_df = display_classification_report_as_dataframe(all_labels, all_predictions)

In [ ]:
plot_confusion_matrix(all_labels, all_predictions, NUM_CLASSES, save_path=confusion_matrix_path)

In [30]:
with open(evaluation_results_path, 'w') as f:
    f.write(f"Test Loss = {avg_test_loss:.4f}\n")
    f.write(f"Test Accuracy = {test_accuracy:.2f}%\n")
    f.write("\nClassification Report:\n")
    f.write(classification_report(all_labels, all_predictions, target_names=[str(i) for i in range(NUM_CLASSES)]))
    f.write("\nPer-Class Accuracy:\n")
    for i, acc in enumerate(per_class_acc):
        f.write(f"Class {i}: {acc:.2f}\n")